# Example: simple pipeline that prints "Hello world!" and produces a .yaml file

#### Define experiment name and base image

These images provide a common starting point for Kubeflow Notebook containers

| Dockerfile       | Container Registry               | Notes                        |
|------------------|----------------------------------|------------------------------|
| [./base](https://github.com/kubeflow/kubeflow/tree/master/components/example-notebook-servers/base)           | [kubeflownotebookswg/base](https://hub.docker.com/r/kubeflownotebookswg/base)         | Common Base Image            |
| [./codeserver](https://github.com/kubeflow/kubeflow/tree/master/components/example-notebook-servers/codeserver)     | [kubeflownotebookswg/codeserver](https://hub.docker.com/r/kubeflownotebookswg/codeserver)  | code-server (Visual Studio Code) |
| [./jupyter](https://github.com/kubeflow/kubeflow/tree/master/components/example-notebook-servers/jupyter)       | [kubeflownotebookswg/jupyter](https://hub.docker.com/r/kubeflownotebookswg/jupyter)     | JupyterLab                   |
| [./rstudio](https://github.com/kubeflow/kubeflow/tree/master/components/example-notebook-servers/rstudio)       | [kubeflownotebookswg/rstudio](https://hub.docker.com/r/kubeflownotebookswg/rstudio)     | RStudio                      |


Kubeflow images extending the base images with common packages used in the real world can be found [here](https://www.kubeflow.org/docs/components/notebooks/container-images/#:~:text=Kubeflow%20provides%20a%20number%20of%20example%20container).

In [7]:
EXPERIMENT_NAME = 'TEST_HELLO'        # Name of the experiment in the UI
BASE_IMAGE = 'kubeflownotebookswg/codeserver-python'

#### Install necessary packages

In [ ]:
!python3 -m pip install kfp

#### Import all necessary Kubeflow packages

In [8]:
import kfp
import kfp.dsl as dsl
from kfp import compiler
from kfp import components

#### Define components of the pipeline
The easiest component is the [Lighweight Python Component](https://www.kubeflow.org/docs/components/pipelines/user-guides/components/lightweight-python-components/), which is built by decorating self-contained python functions with the [**@dsl.component**](https://kubeflow-pipelines.readthedocs.io/en/stable/source/dsl.html#kfp.dsl.component) decorator.

Requirements of a decorated componenent:

1.`Type annotations`: both the function inputs and outputs must have valid KFP [type annotations](https://www.kubeflow.org/docs/components/pipelines/user-guides/data-handling/data-types/) (parameters and artifacts)
* [**Parameters**](https://www.kubeflow.org/docs/components/pipelines/user-guides/data-handling/parameters/) are useful to pass small amount of data between components and when the data created by the component does not represent a machine learning object such as a model, a dataset, or more complex data type. *Valid Parameter annotations include Python built-in int, float, str, bool, typing.Dict, typing.List*
* [**Artifacts**](https://www.kubeflow.org/docs/components/pipelines/user-guides/data-handling/artifacts/) are used to pass complex data or ML objects (Model, Dataset)

2.`Hermetic`: the python function can not reference any symbols or function defined outside of its body.

In [9]:
@dsl.component(base_image=BASE_IMAGE) # The base_image can be extended also using the argument "packages_to_install". Ex: @dsl.component(base_image=BASE_IMAGE, packages_to_install=['numpy==1.21.6'])
def print_hello(
    text: str,
) -> str:
    
    print(text)
    return text

#### Create a pipeline
A pipeline is a definition of a workflow that composes one or more components together. It is built using the [**@dsl.pipeline**](https://kubeflow-pipelines.readthedocs.io/en/stable/source/dsl.html#kfp.dsl.pipeline) decorator.

At runtime, each component execution corresponds to a single container execution, which may create ML artifacts.

In [10]:
@dsl.pipeline
def hello_pipeline(text: str = 'Hello world!') -> str:
    hello_task = print_hello(text=text)
    return hello_task.output

#### Compile and run the pipeline

In [11]:
# Compile pipeline and create yaml file. This can be uploaded to Kubeflow Pipelines instance
compiler.Compiler().compile(hello_pipeline, package_path='hello_pipeline.yaml')

#### Run the pipeline in Kubeflow Dashboard

Follow the instructions at this [link](https://www.kubeflow.org/docs/components/pipelines/user-guides/core-functions/run-a-pipeline/) to run the .yaml file in Kubeflow Dashboard